In [177]:
# NAO USAR A TABELA data_product, E SIM A store_product!!

# Remover primeiro os itens 'C/ 8', verificando se após o 'C/' a string comça com dígito.
# Após isso, remover diretamente os itens com 'C/', por exemplos 'C/8'
# Fazer o mesmo para os itens com a string 'S/' e 'REF'

In [178]:
import pandas as pd
import re

In [179]:
df = pd.read_csv('inputs/data_product_202208091047.csv', usecols = ['id', 'barcode', 'name'])

In [180]:
df.shape

(69111, 3)

In [181]:
df['name'].nunique()


48879

In [182]:
df.drop_duplicates(subset = 'name', inplace = True)

In [183]:
df.shape

(48879, 3)

In [184]:
df['name'] = df['name'].astype(str)

In [185]:
df['name'] = df['name'].str.upper()

In [186]:
df['cleaned'] = df['name']

In [187]:
def remove_startswith(name):
    return lambda words: words.replace(''.join([x for x in words.split(' ') if x.startswith(name)]), '')

In [188]:
def remove_endswith(name, numbers):
    return lambda words: words.replace(''.join([x for x in words.split(' ') if x.endswith(name) and x[:numbers].isdigit()]), '')

In [189]:
def remove_metrics(name):
    to_remove = []
    
    for word in name.split():
        if word[0].isdigit() and 'X' in word and word[-1] == 'M' :
            to_remove.append(word)

    if len(to_remove) > 0:
        return ''. join([name.replace(x, '') for x in to_remove])
    
    return name

In [190]:
# # name = 'BIOSLIM PLUS 4 90CAP'
# name = 'ACTISORB PLUS 10.5X10.5CM MAP105'

# to_remove = []
    
# for word in name.split():
#     if word[0].isdigit() and 'X' in word and word[-1] == 'M':
#         # print('here')
#         to_remove.append(word)

# # print(to_remove)
# if len(to_remove) > 0:
#     print(''. join([name.replace(x, '') for x in to_remove]))
# else:
#     print(name)

# # for item in to_remove:
# #     name = name.replace(item, '')

# # print(name)

In [191]:
startswith_ = ['(', 'REF', 'C/', 'S/', 'INTRAV']

In [192]:
for name in startswith_:
    df['cleaned'] = df['cleaned'].apply(remove_startswith(name))

In [193]:
endswith_ = {
    'G':    -len('G'),
    'L':    -len('L'),
    'UN':   -len('UN'),
    'ML':   -len('ML'),
    'GR':   -len('GR'),
    'CPS':  -len('CPS'),
    'VOL':  -len('VOL'),
    'CAP':  -len('CAP')
}

In [194]:
for k, v in endswith_.items():
    df['cleaned'] = df['cleaned'].apply(remove_endswith(k, v))

In [195]:
df['cleaned'] = df['cleaned'].apply(lambda x: remove_metrics(x))

In [196]:
df['cleaned'] = df['cleaned'].str.strip()

In [197]:
df['num_words'] = df['cleaned'].apply(lambda x: len(str(x).split(' ')))

In [198]:
items_cleaned = [
    48, 57, 58, 60, 76, 77, 82, 83, 84, 88, 94, 102, 106, 116, 118, 122, 171, 174
]

In [199]:
df[df['id'].isin(items_cleaned)].sort_values(by = 'id')

,id,barcode,name,cleaned,num_words
47,48,7896083409235,BIOSLIM PLUS 4 90CAP,BIOSLIM PLUS 4,3
56,57,7891721017377,CR GEL UNIVERSAL FPS20 45G,CR GEL UNIVERSAL FPS20,4
57,58,7891800348361,CURAT CREMER BARBIE 25UN,CURAT CREMER BARBIE,3
59,60,7896040704120,AMACIANTE VIDA MACIA 1L,AMACIANTE VIDA MACIA,3
75,76,7891142142979,EPIDRAT SENSI SH 120ML,EPIDRAT SENSI SH,3
76,77,7898064034801,ABS AFFECTIVE ELASTIC C/10 (FR10),ABS AFFECTIVE ELASTIC,3
81,82,7896025514126,ESC MARCO BONI REF7665T,ESC MARCO BONI,3
82,83,7896025508927,ESC MARCO BONI REF7951,ESC MARCO BONI,3
83,84,7898312327488,ESC PROART BAMBU REF 73252,ESC PROART BAMBU 73252,5
87,88,4015400151715,ABS TAMPAX COMPAK C/8 SUPER,ABS TAMPAX COMPAK SUPER,5


In [200]:
df.sort_values(by = 'cleaned', inplace = True)

In [201]:
df.to_csv('output.csv')

---

In [202]:
df[df['id'].isin([52, 56, 63, 84, 169])]

,id,barcode,name,cleaned,num_words
167,169,7898225526923,ADA TINA INPELE CAPELLI FORTI 60 CPS 30G,ADA TINA INPELE CAPELLI FORTI 60 CPS,7
62,63,7891962004433,"AMANTEIGADO BANANA BAUDUCCO 11,5G","AMANTEIGADO BANANA BAUDUCCO 11,5G",4
51,52,7897371801274,CHUP NUK R 729532,CHUP NUK R 729532,4
55,56,7897371800505,CHUP NUK R729244,CHUP NUK R729244,3
83,84,7898312327488,ESC PROART BAMBU REF 73252,ESC PROART BAMBU 73252,5


In [203]:
# dict_ = {
#     'SAB': 'SABONETE',
#     'DES': 'DESODORANTE',
#     'ESC': 'ESCOVA',
#     'LOC': 'LOCAO',
#     'TRAD': 'TRADICIONAL',
#     'CURAT': 'CURATIVO',
# }